In [1]:
### How it is going

In [2]:
import os
import torch

In [3]:
# read it in to inspect it
os.chdir('/mnt/data/projects/.immune/Personal/Transformers_Tutorial/')
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print(text[:1000])  # print the first 1000 characters to check

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("All the unique characters:", ''.join(chars))
print("Vocab size:", vocab_size)

All the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size: 65


In [6]:
# creating a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: take a list of integers, output a string

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [7]:
# lets encode the entire text into the integer representation
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape,data.type)
print(data[:1000])  # first 1000 characters encoded as integers

torch.Size([1115394]) <built-in method type of Tensor object at 0x7f3531567600>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [8]:
# splitting into train and val sets
n = int(0.9*len(data))  # first 90% will be
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8  # context length for predictions
train_data[:block_size+1] ### till 8 it will be train and 9th character is the target

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
print(decode(train_data[:block_size+1].tolist()))  # checking if decoding works

First Cit


In [11]:
x = train_data[:block_size]  # input
y = train_data[1:block_size+1]  # target
for t in range(block_size):
    context = x[:t+1]  # the context grows with t
    target = y[t]     # the target is always the next character
    print(f'when input is {context} the target: {target}')
    print(f"when input is {decode(context.tolist())!r} the target: {decode([target.tolist()])!r}")

when input is tensor([18]) the target: 47
when input is 'F' the target: 'i'
when input is tensor([18, 47]) the target: 56
when input is 'Fi' the target: 'r'
when input is tensor([18, 47, 56]) the target: 57
when input is 'Fir' the target: 's'
when input is tensor([18, 47, 56, 57]) the target: 58
when input is 'Firs' the target: 't'
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is 'First' the target: ' '
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is 'First ' the target: 'C'
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is 'First C' the target: 'i'
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is 'First Ci' the target: 't'


In [12]:
torch.manual_seed(1337)
batch_size = 4  # parallel sequences processing
block_size = 8  # context length for predictions

train_data.shape, val_data.shape
data = train_data
ix = torch.randint(len(data) - block_size, (batch_size,))
x = torch.stack([data[i:i+block_size] for i in ix])
y - torch.stack([data[i+1:i+block_size+1] for i in ix])
print("x:", x)
print("y:", y)

x: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
y: tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [13]:
torch.manual_seed(1337)
batch_size = 4  # parallel sequences processing
block_size = 8  # context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # random starting indices
    x = torch.stack([data[i:i+block_size] for i in ix])  # input sequences
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # target sequences
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context} the target: {target}')
        print(f"when input is {decode(context.tolist())!r} the target: {decode([target.tolist()])!r}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]) the target: 43
when input is 'L' the target: 'e'
when input is tensor([24, 43]) the target: 58
when input is 'Le' the target: 't'
when input is tensor([24, 43, 58]) the target: 5
when input is 'Let' the target: "'"
when input is tensor([24, 43, 58,  5]) the target: 57
when input is "Let'" the target: 's'
when input is tensor([24, 43, 58,  5, 57]) the target: 1
when input is "Let's" the target: ' '
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
when input is "Let's " the target: 'h'
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
when input i

In [14]:
# Constructing the bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # this is random initialization which will be learned during training.

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # get the predictions
            logits, loss = self(idx) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
# print(targets.shape) # (B*T, C)
# torch.zeroes is for new line character so we can initialize the generation from it.
# This is completely random generation as the model is untrained.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

logits: tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.8786, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [15]:
torch.zeros((1, 1))

tensor([[0.]])

In [16]:
logits.shape

torch.Size([32, 65])

In [17]:
token_embedding=nn.Embedding(vocab_size, vocab_size)
token_embedding.weight

Parameter containing:
tensor([[ 0.6243, -0.1004,  0.2456,  ..., -1.2338,  0.0534, -0.1949],
        [ 0.8603, -0.2675, -0.4354,  ..., -0.1845, -1.0497, -0.8261],
        [ 1.7343,  0.8752,  1.5813,  ...,  1.2927,  0.7182,  0.6097],
        ...,
        [-1.6689, -3.3766, -0.4560,  ...,  0.2780, -1.5953, -0.4737],
        [-1.2969,  1.8702, -1.0598,  ..., -1.2579,  0.1119,  0.3389],
        [-0.0886, -1.0879,  0.9975,  ..., -1.2604,  0.8044, -1.9819]],
       requires_grad=True)

In [18]:
logits = token_embedding(xb) # (B,T,C)

In [19]:
print(logits[0])
print(loss)
print(xb.shape) # B X T
print(yb.shape) # B X T

tensor([[-1.3660e-01, -1.8731e+00,  7.1989e-01,  1.3609e-01,  9.8589e-01,
          1.5483e+00, -2.6414e-01, -1.1776e+00,  5.1714e-01, -4.6053e-01,
          4.2017e-01, -9.6025e-01,  1.3287e+00,  2.0909e-01,  8.6215e-01,
          4.3085e-02,  5.7950e-01, -2.0212e+00, -8.1185e-01,  6.5908e-01,
         -7.5360e-02,  1.3957e+00,  2.0643e+00, -2.0412e+00, -3.7620e-01,
         -3.8499e-01,  3.5262e-01, -4.5423e-01,  1.1916e+00, -8.5033e-01,
         -1.1318e+00,  9.0647e-01,  5.6884e-01, -2.9375e+00,  4.6553e-01,
         -9.6770e-01, -1.4652e+00, -2.6071e-01, -1.0342e+00, -1.3620e+00,
         -1.2595e+00, -1.8476e+00,  1.1339e+00, -2.0390e-01,  2.8299e-01,
          3.1425e-01,  1.5377e+00,  9.7995e-02,  1.5568e-01, -3.9144e-01,
         -1.0180e+00,  2.9127e-01, -1.4972e+00,  8.3221e-01,  1.5373e+00,
         -1.1809e+00,  2.8087e+00,  1.7374e+00, -5.5859e-01, -8.0195e-01,
         -5.1946e-01, -9.9884e-01,  2.1045e-01,  8.7487e-01,  2.4380e-01],
        [ 4.1719e-01,  6.3408e-01, -1

Training the model

In [20]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [21]:
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    # optimize the model
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.6904830932617188


In [22]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ss d tcI:
AUNorthiref s is, XLANzncouso;AUCl
AHllico ithen, Horeo t had,

Mh
FINolly



Uy ou ttirve
' ll ato, s then w; s w
EQzL:CEXR:
Whas nsu bfof; cond,

IFtrune e fo wa at we al
Cid thind ad cas;
Youtod,
Fo s mp aSwhy'lout.
RFXWathininoth itethe t r; terus rs t;
vell COUCENET:
bints tes?$a:
OLk'douid ours;
p' w'raver:

e,
AUA c,
UcousexFIEOr orous sthVIOLinr om!
STEDICHe kenf tis bau d IU horf t q.
bur,
Issthe; rinodvely,V&ven tang, horche.?
Thaceayousomege, thage pe'se.
ORYQ!ve
JFVbep On s


## The mathematical trick in self-attention

In [23]:
## This is for 2 dimesnional matrix multiplication
# Since in transformer we will be using 2D matrices for attention calculations.
# In attention we use only those character before the current character for prediction.
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # Lower triangular matrix
a = a / torch.sum(a, dim=1, keepdim=True) # scaling the rows to sum to 1
print(a)
b = torch.randint(0, 10, (3, 2)).float()
print(b)
c = a @ b
print(c) 


tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [24]:
## This is for 2 dimesnional matrix multiplication
torch.manual_seed(42)
a = torch.tril(torch.ones(3,4,3)) # Lower triangular matrix
a = a / torch.sum(a, dim=1, keepdim=True) # scaling the rows to sum to 1
# print(a)
print(a.shape)
b = torch.randint(0, 10, (3, 2)).float()
# print(b)
print(b.shape)
c = a @ b
# print(c)
print(c.shape)
# so regarding the dimensions, a = (3,4,3) and b = (3,2) results in c = (3,4,2)
# since b = (3,3,2) and while matrix multiplication the last two dimensions are considered for multiplication.
# so a = (4,3) and b = (3,2) results in c = (4,2)


torch.Size([3, 4, 3])
torch.Size([3, 2])
torch.Size([3, 4, 2])


In [25]:
# try out on our dataset
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B,T,C)
x.shape
print(x[0])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])


In [26]:
### In the self attention here we are just averaging the previous tokens to get a sense of context.

xbow = torch.zeros((B, T, C))
print(xbow.shape)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev,0) # averaging the previous tokens including the current token
print(xbow.shape)
print(xbow[0])

torch.Size([4, 8, 2])
torch.Size([4, 8, 2])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [27]:
### Another more efficient way
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)  #
print(wei)
xbow2 = wei @ x  # (T,T) @ (T,C) -->
print(xbow2.shape)
print(xbow2[1])
print(xbow[1])
torch.allclose(xbow[0], xbow2[0]) ### it is same

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
torch.Size([4, 8, 2])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0.0020],
        [ 0.0712, -0.1128],
        [ 0.2527,  0.2149]])
tensor([[ 1.3488, -0.1396],
        [ 0.8173,  0.4127],
        [-0.1342,  0.4395],
        [ 0.2711,  0.4774],
        [ 0.2421,  0.0694],
        [ 0.0084,  0

True

In [28]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [29]:
wei = torch.zeros((T,T))
wei

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [30]:
### how much does the previous elements contribute to the current element. 
# so we can use the softmax which is kind of the normalization
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)
wei = F.softmax(wei, dim = -1)
wei

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [31]:
# Constructing the bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        token_emb = self.token_embedding_table(idx)  # (B,T,C)
        logits = self.lm_head(token_emb)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # get the predictions
            logits, loss = self(idx) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 32
batch_size = 32
m = BigramLanguageModel()
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
# print(targets.shape) # (B*T, C)
# torch.zeroes is for new line character so we can initialize the generation from it.
# This is completely random generation as the model is untrained.
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

logits: tensor([[ 0.6532, -0.5768,  0.1623,  ...,  0.9086, -0.3203, -0.5369],
        [ 0.4420,  0.1231,  0.8807,  ...,  0.5617,  0.1265,  0.2913],
        [ 1.0582, -0.5045,  0.3036,  ..., -0.3142,  0.5471,  0.7962],
        ...,
        [-0.1580, -0.4475,  0.3257,  ...,  0.1921,  0.7424,  0.4570],
        [ 0.4546, -0.3635,  0.2983,  ...,  0.2652,  0.5731,  0.0840],
        [-0.6873, -0.4842,  0.6730,  ..., -0.5416, -0.7946, -0.7934]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.2468, grad_fn=<NllLossBackward0>)
torch.Size([32, 65])

lN!BJ'kysLCMFJPKOL?DP-QWwrEoL?jLDJQOL.f'RIHD'Hdhs Yv,wxatnscMZwtEOS'palkq3ssZeAvzF-QT;eMk;x.gQSFCLgx


In [32]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [33]:
## Training a model
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5342628955841064


In [34]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Th thalldy hangilyoteng h hasbe pan hatrance
RDe hicomyonthar's
PES:
AKEd ith henoungincenonthiousir thondy, y heltieiengerofo'dsssit ey
KINld pe wither vouprrouthercckeha t,
K:

My hind tt hinig t ouchos tes; st younind wotte grotonear 'so it t jod weancothanan hay. t--s n prids, r loncave w hollular e O:
HIs; ht anje caike ineent.

Lavinde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, aridesar, wyonthenous aves, theresseys
Plorseelapinghiybend yof GLANCHI me. strsithisgothers w de o! ABe


#### Adding the Position Embedding to it

In [37]:
xb.shape

torch.Size([32, 8])

In [53]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

## First lets initialize 
vocab_size = 65 # since in the Shakespeare vocabulary has 65 characters 
emb_size = 128 # total number of dimensions
batch_size = 32 # batch size
token_size = 8
## It is just randomly creating an embedding for all the vocabulary and Tokens with 128 dimensions
token_embedding_table = nn.Embedding(vocab_size, emb_size) 
token_position_table = nn.Embedding(token_size, emb_size)
lm_head = nn.Linear(emb_size, vocab_size) 

In [61]:
print("character embedding")
print(token_embedding_table)
print(token_embedding_table.weight[0,:].shape)
# print(token_embedding_table.weight[0,:] )
# This shows for first character what is the weight will keep on updating during the backpropogation
print("position embedding")
print(token_position_table)
print(token_position_table.weight[0,:].shape)
# print(token_position_table.weight[0,:])
# This shows for first token position what is the weight will keep on updating during the backpropogation
print("linear Model")
print(lm_head)
print(lm_head.weight[0,:].shape)
# lm_head.weight[0,:]


character embedding
Embedding(65, 128)
torch.Size([128])
position embedding
Embedding(8, 128)
torch.Size([128])
linear Model
Linear(in_features=128, out_features=65, bias=True)
torch.Size([128])


In [113]:
parameters = [
        token_embedding_table.weight,
        token_position_table.weight,
        lm_head.weight,
        lm_head.bias
        ]


In [125]:
### After initialization, we will input the values
# print("Input=",xb[0:1:10], xb.shape,"\nTarget=",yb[0:1:10], yb.shape)
# Now we have to embed these tokens
# token_character embedding which will provide us Batch, Token, Embedding 32, 8, 128 (B,T,C)

for steps in range(10000):
    B,T = xb.shape
    token_emb= token_embedding_table(xb)
    # print("Input_embdding=",token_emb.shape)
    position_emb= token_position_table(torch.arange(T)) ## Since we have to take care of the position
    # print("Input_pos_embdding=",position_emb.shape)
    x = token_emb + position_emb
    # print("x=",x.shape)
    logits = lm_head(x) # xW.transpose() + bias
    B,T,C = logits.shape
    logits3 = logits.view(B*T,C)
    # print("logits:",logits3.shape)
    targets = yb.view(B*T)
    # print('targets:',targets.shape)
    loss = F.cross_entropy(logits3, targets)
    # optimizer.zero_grad(set_to_none = True)
    # loss.backward()
    # optimizer.step()
    # Gradient reset
    for p in parameters:
        p.grad = None
    loss.backward()
    # Update
    for p in parameters:
        p.data += -lr * p.grad
    # logits2 = logits[:,-1,:]
    # print("logits=",logits2.shape)
    # prob = F.softmax(logits2, dim = -1)
    # print("prob=",prob.shape)
    # print(prob.sum(1))

print(loss.item())

1.0955548286437988


In [155]:
def generate(idx, max_new_tokens):
    # idx: (B, T) tensor of token indices (context). Usually start with [[0]].
    for _ in range(max_new_tokens):
        B, T = idx.shape
        # Forward pass for only the current context
        token_emb = token_embedding_table(idx)                         # (B, T, C)
        position_emb = token_position_table(torch.arange(T)) # (T, C)
        x = token_emb + position_emb                                   # (B, T, C)
        logits = lm_head(x)                                            # (B, T, vocab)
        # take only the last time step
        logits = logits[:, -1, :]                                      # (B, vocab)
        # convert logits → probabilities
        probs = F.softmax(logits, dim=-1)                              # (B, vocab)
        # sample next token
        idx_next = torch.multinomial(probs, num_samples=1)             # (B, 1)
        # append to sequence
        idx = torch.cat((idx, idx_next), dim=1)                        # (B, T+1)
    return idx


In [157]:
start = torch.zeros((1, 1), dtype=torch.long)   # context = "unknown token" / start token
output = generate(start, max_new_tokens=8)
print(decode(output[0].tolist()))


KATHAnor


In [158]:
context = torch.zeros((1, 1), dtype=torch.long)  # or start token / first character
generated = context.clone()

for _ in range(200):   # generate 200 characters
    # crop context to last 8 chars (model's block size)
    context_cropped = generated[:, -8:]

    token_emb = token_embedding_table(context_cropped)
    pos_emb = token_position_table(torch.arange(context_cropped.shape[1]))
    x = token_emb + pos_emb
    logits = lm_head(x)

    # take logits of last time step
    logits_last = logits[:, -1, :]
    probs = F.softmax(logits_last, dim=-1)

    # sample from distribution
    next_token = torch.multinomial(probs, num_samples=1)

    # append to generated sequence
    generated = torch.cat([generated, next_token], dim=1)

print(generated)


tensor([[ 0, 23, 13, 45, 43, 56, 60, 39, 47, 49, 43,  1, 61, 47, 49, 43,  1, 61,
         47, 49, 43,  1, 46, 39, 52, 51, 39, 47, 49, 43, 52, 41, 47, 49, 43, 57,
         43, 56, 57,  1, 63, 43,  1, 51, 39, 52, 49, 43, 57,  1, 61, 47, 49, 43,
          1, 51, 39, 47, 49, 43, 57, 47, 49, 43,  1, 61, 47, 49, 43,  1, 41, 46,
         63,  1, 59, 54, 53, 51, 39, 52, 53, 52, 41, 47, 49, 43, 56, 60, 39, 52,
         41, 39, 48,  1, 39, 52, 41, 47, 49, 43, 57, 43, 57, 43,  1, 61, 47, 49,
         43, 58, 46, 47, 49, 43,  1, 52, 53, 58, 63,  1, 51, 39, 52, 41, 47, 49,
         43,  1, 46, 39, 52, 41, 47, 49, 43,  1, 44, 44, 44, 43, 52, 58, 39, 52,
         53, 52, 53, 49, 43,  1, 44, 44, 56, 47, 49, 43,  1, 61, 47, 49, 43,  1,
         41, 47, 49, 43,  1, 59, 54, 53, 51, 59, 54, 53, 56, 53, 58, 63, 43,  1,
         59, 54, 53, 51, 39, 52, 58,  1, 44, 44, 44, 44, 43,  1, 51, 39, 52, 58,
         39, 52, 58]])


In [159]:
decoded = ''.join([itos[int(i)] for i in generated[0].tolist()])
print(decoded)



KAgervaike wike wike hanmaikencikesers ye mankes wike maikesike wike chy upomanoncikervancaj ancikesese wikethike noty mancike hancike fffentanonoke ffrike wike cike upomuporotye upomant ffffe mantant


In [121]:
for step in range(10000):
    B, T = xb.shape
    token_emb = token_embedding_table(xb)                        # (B, T, C)
    position_emb = token_position_table(torch.arange(T))  # (T, C)
    x = token_emb + position_emb                                 # (B, T, C)

    logits = lm_head(x)                                          # (B, T, vocab)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = yb.view(B*T)

    loss = F.cross_entropy(logits, targets)

    # Gradient reset
    for p in parameters:
        p.grad = None

    # Backward
    loss.backward()

    # Update
    lr = 1e-3
    for p in parameters:
        p.data += -lr * p.grad

print(loss.item())


1.1360057592391968


In [ ]:
## Training a model
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
B,T,C = logits.shape
logits3 = logits.view(B*T,C)
print("logits:",logits3.shape)
targets = yb.view(B*T)
print('targets:',targets.shape)
F.cross_entropy(logits3, targets)

logits: torch.Size([256, 65])
targets: torch.Size([256])


tensor(4.4113, grad_fn=<NllLossBackward0>)

In [ ]:
# Constructing the bigram model
## In the Bigram model it does not really matter the position since you will only be contructing or predicting based on the one character before
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb) # this is random initialization which will be learned during training.
        self.token_position_embedding_table = nn.Embedding(block_size, n_emb) # position embedding of one batch in the context
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers where idx is xb and targets is yb
        B,T = idx.shape # B-Batch, T-Tokens, C-Channel i.e. dimensions
        token_emb = self.token_embedding_table(idx)  # (B,T,C) (32,8,32)
        position_emb = self.token_position_embedding_table(torch.arange(T)) ## for each block is arrange as 0,1,2,3
        x = token_emb + position_emb
        logits = self.lm_head(x) # xW.transpose() + b 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens): # Throw-away variable in loops so it does not fill the variable memory.
            # get the predictions
            logits, loss = self(idx) # Calls the Forward method
            # focus only on the last time step
            logits = logits[:, -1, :]  # (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B,1) #  draws a sample from the probability distribution. It returns the index of the next chosen token.
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
        return idx

n_emb = 32 # this is basically the dimensions size
batch_size = 32
m = BigramLanguageModel()
logits, loss = m(xb, yb) # PyTorch overloads the __call__() method of nn.Module. This calls the forward() method. 
print("logits:", logits)
print("loss:", loss)
print(logits.shape) # (B*T, C)
# print(targets.shape) # (B*T, C)
# torch.zeroes is for new line character so we can initialize the generation from it.
# This is completely random generation as the model is untrained.


logits: tensor([[-0.0929, -1.3390, -1.8530,  ...,  0.7327, -0.0828,  0.3238],
        [ 0.6940, -0.1655,  0.6275,  ..., -0.1051, -0.2384, -0.8234],
        [ 0.6146,  1.3971,  0.7296,  ...,  0.5760,  0.8021,  0.4972],
        ...,
        [ 1.0967,  0.7229,  0.1110,  ...,  0.0170,  1.9136,  0.7253],
        [ 0.9462, -0.4075, -0.4941,  ..., -0.2246,  0.6021,  0.7631],
        [-0.2758, -0.7347, -0.0909,  ..., -0.9703,  0.7193, -0.0722]],
       grad_fn=<ViewBackward0>)
loss: tensor(4.4489, grad_fn=<NllLossBackward0>)
torch.Size([256, 65])


In [36]:
## Self Attention Mechanism, we require key, query, and values
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch time channels
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x)
q = query(x)

wei = q @ k.transpose(-2,-1)
tril = torch.tril(torch.ones(T,T))

wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei[0])
wei = F.softmax(wei, dim = -1)
print(wei[0])

v = value(x)
out = wei @ v
print(out[0])
out.shape

tensor([[-1.7629,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-3.3334, -1.6556,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-1.0226, -1.2606,  0.0762,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.7836, -0.8014, -0.3368, -0.8496,    -inf,    -inf,    -inf,    -inf],
        [-1.2566,  0.0187, -0.7880, -1.3204,  2.0363,    -inf,    -inf,    -inf],
        [-0.3126,  2.4152, -0.1106, -0.9931,  3.3449, -2.5229,    -inf,    -inf],
        [ 1.0876,  1.9652, -0.2621, -0.3158,  0.6091,  1.2616, -0.5484,    -inf],
        [-1.8044, -0.4126, -0.8306,  0.5898, -0.7987, -0.5856,  0.6433,  0.6303]],
       grad_fn=<SelectBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.02

torch.Size([4, 8, 16])